## Attributi sull'intero testo

- Dimensione del vocabolario (V)
- Lunghezza del testo in numero di parole (T)
- Rapporto V/T
- Entropia (H)

In [1]:
import math

def word_counter(RDD):
    '''
    Data una RDD, conta quante volte compare ogni parola ritornando anche la dimensione del vocabolario.
    
    Parameters
    ----------
    RDD : RDD
        RDD del file in input

    Returns
    -------
    (RDD, int)
        RDD dell'output del word count e dimensione del vocabolario
    '''
    
    word_counter = (RDD.flatMap(lambda x: x)
                    .map(lambda x: (x,1))
                    .reduceByKey(lambda a,b: a+b)
                    .sortBy(lambda x: -x[1])
                   )
    
    return word_counter, word_counter.count()

def text_length_in_words(RDD_word_counter):
    '''
    Calcola la lunghezza del testo in termini di numero di parole.
    
    Parameters
    ----------
    RDD_word_counter : RDD
        RDD dell'output del word count

    Returns
    -------
    int
        numero di parole totali presenti nel testo
    '''

    return (RDD_word_counter.map(lambda x: x[1])
            .reduce(lambda a,b: a+b)
           )

def entropy(RDD_word_counter, text_len):
    '''
    Calcola l'entropia (numero medio di bit richiesti per rappresentare tutte le parole del testo).
    
    Parameters
    ----------
    RDD_word_counter : RDD
        RDD dell'output del word count
    text_len : int
        numero di parole totali presenti nel testo

    Returns
    -------
    float
        valore dell'entropia
    '''
    
    return -(RDD_word_counter.map(lambda x: (x[1]/text_len) * math.log2(x[1]/text_len))
             .reduce(lambda a,b: a+b)
            ) # l'entropia ha segno negativo

## Attributi sulle frasi

- Lunghezza massima di una frase (MSL)
- Lunghezza media di una frase (ASL)
- Lunghezza minima di una frase (mSL)
- Distribuzione di probabilità delle lunghezze delle frasi (PDSL)
- Probabilità della lunghezza di frase più frequente (pMFSL)

In [2]:
def sentence_lengths(RDD):
    '''
    Calcola le lunghezze (in termini di numero di parole) di tutte le frasi di un testo.
    
    Parameters
    ----------
    RDD : RDD
        RDD del file in input

    Returns
    -------
    RDD
        RDD che contiene le lunghezze delle frasi
    '''
    
    # operazioni preliminari sul testo
    text = RDD.flatMap(lambda x: x).reduce(lambda a,b: a + ' ' + b) # metto tutto il testo in una stringa unica
    text = text.replace("?", ".") # ? termina una frase
    text = text.replace("!", ".") # ! termina una frase
    text = text.split('. ') # splitto quando trovo un carattere che termina una frase (. seguito da uno spazio)
    
    return (sc.parallelize(text)
            .map(lambda x: len(x.split(' ')))
           ) # per ogni frase trovata conto le sue parole

def prob_distr_of_sentence_length(RDD_sen_len):
    '''
    Ritorna una lista che contiene la distribuzione di probabilità delle lunghezze delle frasi.
    
    Parameters
    ----------
    RDD_sen_len : RDD
        RDD dell'output di sentence_lengths

    Returns
    -------
    list
        distribuzione di probabilità delle lunghezze delle frasi
    '''
    
    tot = RDD_sen_len.count()

    return (RDD_sen_len.map(lambda x: (x,1))
            .reduceByKey(lambda a,b: a+b)
            .map(lambda x: (x[0], x[1]/tot))
            .sortBy(lambda x: -x[1])
           )

## Attributi sulla probabilità delle parole

- Distribuzione di probabilità delle 30 parole più comuni (PkMCW)
- Probabilità della parola più comune escludendo 'and' e 'the' (pMCW)
- Probabilità della parola più comune escludendo articoli e preposizioni (pMCWx)
- Probabilità della parola 'the' (pThe)
- Probabilità della virgola (pComma)

In [3]:
def prob_distr_of_most_common_words(RDD_word_counter, text_len):
    '''
    Ritorna la distribuzione di probabilità delle parole più comuni.
    
    Parameters
    ----------
    RDD_word_counter : RDD
        RDD dell'output del word count
    text_len : int
        numero di parole totali presenti nel testo

    Returns
    -------
    RDD
        RDD che contiene la distribuzione di probabilità
    '''
    
    return RDD_word_counter.map(lambda x: (x[0], x[1]/text_len))

def prob_of_the_most_common_word(RDD_prob_distr_of_MCWs):
    '''
    Ritorna la probabilità della parola più comune (escludendo 'and' e 'the').
    
    Parameters
    ----------
    RDD_prob_distr_of_MCWs : RDD
        RDD con la distribuzione di probabilità
    
    Returns
    -------
    tuple
        MCW e relativa probabilità
    '''
    
    return (RDD_prob_distr_of_MCWs
            .filter(lambda x: x[0] != "and" and x[0] != "the")
            .take(1)
           )[0]

def prob_of_the_most_common_word_x(RDD_prob_distr_of_MCWs):
    '''
    Ritorna la probabilità della parola più comune (escludendo articoli e preposizioni).
    
    Parameters
    ----------
    RDD_prob_distr_of_MCWs : RDD
        RDD con la distribuzione di probabilità
    
    Returns
    -------
    tuple
        MCWx e relativa probabilità
    '''
    
    prep_art = open("preposizioni_e_articoli.txt").read().splitlines()
    
    return (RDD_prob_distr_of_MCWs
            .filter(lambda x: x[0] not in prep_art)
            .take(1)
           )[0]

def prob_of_The(RDD_prob_distr_of_MCWs):
    '''
    Ritorna la probabilità della parola "the".
    
    Parameters
    ----------
    RDD_prob_distr_of_MCWs : RDD
        RDD con la distribuzione di probabilità
    
    Returns
    -------
    RDD
        RDD che contiene la probabilità della parola "the"
    '''
    
    return (RDD_prob_distr_of_MCWs
            .filter(lambda x: x[0] == "the")
           )

def prob_of_comma(RDD_sentences_data, text_len):
    '''
    Ritorna la probabilità di presenza della virgola.
    
    Parameters
    ----------
    RDD_sentences_data : RDD
        RDD del file in input
    text_len : int
        numero di parole totali presenti nel testo

    Returns
    -------
    int
        probabilità di presenza della virgola
    '''
    
    return (RDD_sentences_data
            .flatMap(lambda x: x)
            .filter(lambda x: "," in x)
            .count()
           ) / text_len

## Attributi sulla distanza

- Distanza media/minima/massima tra apparenze consecutive della MCW (adMCW, mdMCW, MsMCW)
- Distanza media/minima/massima tra apparenze consecutive della MCW escludendo articoli e preposizioni (adMCWx, mdMCWx, MsMCWx)
- Distanza media/minima/massima tra apparenze consecutive di 'the' (adThe, mdThe, MsThe)
- Distanza media/minima/massima tra apparenze consecutive della virgola (adComma, mdComma, MsComma)

In [4]:
def distance_consec_appear(RDD, word):
    '''
    Ritorna una lista che contiene le distanze tra apparenze consecutive di word.
    
    Parameters
    ----------
    RDD : RDD
        RDD del file in input
    word : str
        parola da trattare

    Returns
    -------
    list
        distanze tra apparenze consecutive di word
    '''
    
    if word == ',':
        vect_pos = (RDD.flatMap(lambda x:x)
                    .zipWithIndex()
                    .filter(lambda x: ',' in x[0])
                    .map(lambda x: x[1])
                    .collect()
                   )
    else:
        vect_pos = (RDD.flatMap(lambda x:x)
                    .zipWithIndex()
                    .filter(lambda x: x[0] == word)
                    .map(lambda x: x[1])
                    .collect()
                   )
    
    vect_dis = []
    
    for i in range(1, len(vect_pos)):
        vect_dis.append(vect_pos[i] - vect_pos[i-1])
    
    return vect_dis

## Funzioni di supporto

In [5]:
def remove_number_some_punctuation_marks(row):
    '''
    Rimuove i caratteri numerici e i caratteri " e -- dalla stringa e la trasforma in lower-case.

    Parameters
    ----------
    row : str
        riga del file in input

    Returns
    -------
    res : str
        riga senza --, ", e i numeri
    '''
    
    lowercase = row.lower()
    lowercase = lowercase.replace("--", " ")
    
    res = ""
    
    for char in lowercase:
        if not ('0' <= char <= '9' or char == '"'):
            res += char

    return res

def remove_number_punctuation_marks(row):
    '''
    Rimuove i caratteri numerici e i segni di punteggiatura dalla stringa e la trasforma in lower-case.

    Parameters
    ----------
    row : str
        riga del file in input

    Returns
    -------
    res : str
        riga senza numeri e segni di punteggiatura
    '''
    
    lowercase = row.lower()
    lowercase = lowercase.replace("--", " ")
    
    res = ""
    
    for char in lowercase:
        if 'a' <= char <= 'z' or char == ' ' or char == '-' or char == "'":
            res += char

    return res

def load_file_without_punctuations_marks(filepath):
    '''
    Carica il contenuto di un file in una RDD (tralasciando numeri e segni di punteggiatura).
    
    Parameters
    ----------
    filepath : str
        path del file da caricare

    Returns 
    -------
    RDD
        RDD del contenuto del file
    '''
    
    # caricamento del dataset
    raw_text = sc.textFile(filepath)

    # rimuoviamo i numeri e i segni di punteggiatura
    
    return (raw_text.filter(bool)                    # rimuoviamo le stringhe vuote
        .map(remove_number_punctuation_marks)
        .map(lambda x : ' '.join(x.split()))        # rimuoviamo diversi spazi bianchi con uno
        .map(lambda row : row.split(" "))
       )

def load_file_without_number(filepath):
    '''
    Carica il contenuto di un file in una RDD (tralasciando i numeri e i caratteri " e --).
    
    Parameters
    ----------
    filepath : str
        path del file da caricare

    Returns 
    -------
    RDD
        RDD del contenuto del file
    '''
    
    # caricamento del dataset
    raw_text = sc.textFile(filepath)

    # rimuoviamo i numeri e i segni di punteggiatura
    
    return (raw_text.filter(bool)                    # rimuoviamo le stringhe vuote
        .map(remove_number_some_punctuation_marks)
        .map(lambda x : ' '.join(x.split()))        # rimuoviamo diversi spazi bianchi con uno
        .map(lambda row : row.split(" "))
       )

def getCollection(RDD):
    return RDD.collect()

def getValue(RDD):
    return RDD.collect()[0]

def mean_std_couple(_list, tot_el):
    for i in range(0, tot_el - len(_list)):
        _list.append(0)

    return (statistics.mean(_list), statistics.stdev(_list))

## Salvataggio degli attributi

In [34]:
import pickle
import statistics

def generate_metrics(file_in):
    '''
    Ritorna un dizionario contenente tutti gli attributi estratti da un testo.
    
    Parameters
    ----------
    file_in : str
        path del file da analizzare
    
    Returns
    -------
    dict
        dizionario degli attributi estratti
    '''
    
    res = {}
    
    # consideriamo il testo SENZA i segni di punteggiatura
    print("Caricamento del file in memoria ...", end=" ")
    data = load_file_without_punctuations_marks(file_in)
    data.persist()
    print("caricamento completato")

    # calcoliamo le prime metriche
    print("Calcolo delle prime metriche, attendere ...", end=" ")
    
    RDD_word_counter, vocabulary_size = word_counter(data)
    RDD_word_counter.persist()
    text_length = text_length_in_words(RDD_word_counter)
    entropy_value = entropy(RDD_word_counter, text_length)
    
    RDD_prob_distr_of_MCWs = prob_distr_of_most_common_words(RDD_word_counter, text_length)
    prob_the_most_common_word = prob_of_the_most_common_word(RDD_prob_distr_of_MCWs)
    prob_the_most_common_word_x = prob_of_the_most_common_word_x(RDD_prob_distr_of_MCWs)
    prob_the = prob_of_The(RDD_prob_distr_of_MCWs)

    MCW = prob_the_most_common_word[0]
    dist_consec_MCW = distance_consec_appear(data, MCW)
    
    MCWx = prob_the_most_common_word_x[0]
    dist_consec_MCWx = distance_consec_appear(data, MCWx)
    
    dist_consec_the = distance_consec_appear(data, 'the')
        
    print("calcolo completato")

    
    # consideriamo il testo CON i segni di punteggiatura
    print("Caricamento del file in memoria ...", end=" ")
    sentences_data = load_file_without_number(file_in)
    sentences_data.persist()
    print("caricamento completato")
    
    # calcoliamo altre metriche
    print("Calcolo di ulteriori metriche, attendere ...", end=" ")
    
    RDD_sen_lengths = sentence_lengths(sentences_data)
    RDD_sen_lengths.persist()

    sen_lengths = RDD_sen_lengths.collect()

    prob_distr_freq_sen = prob_distr_of_sentence_length(RDD_sen_lengths)
    
    p_comma = prob_of_comma(sentences_data, text_length)

    dist_consec_comma = distance_consec_appear(sentences_data, ',')
    
    print("calcolo completato")
    
    
    # popoliamo il dizionario
    
    # attributi sull'intero testo
    res['vocabulary_size'] = vocabulary_size
    res['text_length'] = text_length
    res['V_T'] = vocabulary_size/text_length
    res['entropy'] = entropy_value
    
    # attributi sulle frasi
    res['avg_sentence_len'] = sum(sen_lengths)/len(sen_lengths)
    res['max_sentence_len'] = max(sen_lengths)
    #res['min_sentence_len'] = min(sen_lengths)
    res['prob_distr_freq_sen'] = getCollection(prob_distr_freq_sen)
    res['prob_most_freq_sen'] = getValue(prob_distr_freq_sen)[1]
    
    # attributi sulla probabilità delle parole
    res['prob_distr_of_30'] = RDD_prob_distr_of_MCWs.take(30)
    res['prob_of_the_most_common_word'] = prob_the_most_common_word[1]
    res['prob_of_the_most_common_word_x'] = prob_the_most_common_word_x[1]
    res['prob_of_the'] = getValue(prob_the)[1]
    res['prob_of_comma'] = p_comma
    
    # attributi sulla distanza
    res['avg_dist_consec_comma'] = sum(dist_consec_comma)/len(dist_consec_comma)
    res['min_dist_consec_comma'] = min(dist_consec_comma)
    res['max_dist_consec_comma'] = max(dist_consec_comma)
    
    res['avg_dist_consec_MCW'] = sum(dist_consec_MCW)/len(dist_consec_MCW)
    res['min_dist_consec_MCW'] = min(dist_consec_MCW)
    res['max_dist_consec_MCW'] = max(dist_consec_MCW)
    
    res['avg_dist_consec_MCWx'] = sum(dist_consec_MCWx)/len(dist_consec_MCWx)
    res['min_dist_consec_MCWx'] = min(dist_consec_MCWx)
    res['max_dist_consec_MCWx'] = max(dist_consec_MCWx)
    
    res['avg_dist_consec_the'] = sum(dist_consec_the)/len(dist_consec_the)
    res['min_dist_consec_the'] = min(dist_consec_the)
    res['max_dist_consec_the'] = max(dist_consec_the)
    
    return res
    
def save_metrics(file_in, file_out):
    '''
    Salva un dizionario in un file.
    
    Parameters
    ----------
    file_in : str
        path del file da analizzare
    file_out: str
        path del file in cui salvare la entry

    Returns
    -------
    None
    '''

    entry = generate_metrics(file_in)
    
    with open(file_out, 'ab') as fout:
        pickle.dump(entry, fout, pickle.HIGHEST_PROTOCOL)
    
    print("Salvataggio completato")

def load_metrics(file_in):
    '''
    Carica i dizionari degli attributi presenti in un file.
    
    Parameters
    ----------
    file_in : str
        path del file da cui caricare

    Returns
    -------
    list
        lista di dizionari degli attributi
    '''

    res = []
    
    with open(file_in, "rb") as fin:
        while True:
            try:
                res.append(pickle.load(fin))
            except EOFError:
                break
                
    return res

def author_metrics(author_name):
    '''
    Calcola media e deviazione standard degli attributi (provenienti da vari testi) di un autore.
    
    Parameters
    ----------
    author_name : str
        nome del file contenente i dizionari degli attributi

    Returns
    -------
    dict
        dizionario con media e deviazione standard degli attributi
    '''
    
    analisi = load_metrics(author_name)
    res = {}
    
    # recupero gli attributi dai dizionari e li metto sotto la stessa chiave
    for diz in analisi:
        for key in diz:
            try:
                res[key] += [diz[key]]
            except:
                res[key] = [diz[key]]
    
    for key in res:
        if type(res[key][0]) != list:
            # se l'attributo NON è una lista, calcolo direttamente media e deviazione standard
            res[key] = (statistics.mean(res[key]), statistics.stdev(res[key]))
        else:
            # se l'attributo è una lista, calcolo direttamente media e deviazione standard
            tot_el = len(res[key])
            res[key] = (sc.parallelize(res[key])
                        .flatMap(lambda x: x)
                        .map(lambda x: (x[0], [x[1]]))
                        .reduceByKey(lambda a,b: a+b)
                        .map(lambda x: (x[0], mean_std_couple(x[1], tot_el)))
                        .collect()
                       )
    
    return res

# Controllo attributi

In [10]:
data = load_file_without_punctuations_marks("datasets/Anthony Trollope___The O'Conors of Castle Conor from Tales from all Countries.txt")
data.persist()

PythonRDD[8] at RDD at PythonRDD.scala:53

In [29]:
RDD_word_counter, vocabulary_size = word_counter(data)
RDD_word_counter.persist() # ci servirà per calcolare altri parametri
print("V:\t", vocabulary_size)

text_length = text_length_in_words(RDD_word_counter)
print("T:\t", text_length)

print("V/T:\t", vocabulary_size/text_length)

entropy_value = entropy(RDD_word_counter, text_length)
print("H:\t", entropy_value)

V:	 1558
T:	 7653
V/T:	 0.20358029530902913
H:	 8.605861060321123


In [30]:
sentences_data = load_file_without_number("datasets/Anthony Trollope___The O'Conors of Castle Conor from Tales from all Countries.txt")
sentences_data.persist()

PythonRDD[172] at RDD at PythonRDD.scala:53

In [33]:
RDD_sen_lengths = sentence_lengths(sentences_data)
RDD_sen_lengths.persist() # ci servirà per calcolare altri parametri

sen_lengths = RDD_sen_lengths.collect()
print("MSL:\t", max(sen_lengths))
print("ASL:\t", sum(sen_lengths)/len(sen_lengths))
print("mSL:\t", min(sen_lengths))

prob_distr_freq_sen = prob_distr_of_sentence_length(RDD_sen_lengths)
print("PDSL:\t", getCollection(prob_distr_freq_sen))
print("pMFSL:\t", getValue(prob_distr_freq_sen))

MSL:	 84
ASL:	 14.862135922330097
mSL:	 1
PDSL:	 [(7, 0.06407766990291262), (2, 0.05631067961165048), (9, 0.04660194174757282), (10, 0.04660194174757282), (17, 0.04271844660194175), (12, 0.040776699029126215), (8, 0.038834951456310676), (16, 0.038834951456310676), (1, 0.038834951456310676), (3, 0.038834951456310676), (15, 0.038834951456310676), (6, 0.036893203883495145), (11, 0.03495145631067961), (5, 0.03495145631067961), (13, 0.03300970873786408), (4, 0.031067961165048542), (20, 0.02912621359223301), (25, 0.02330097087378641), (22, 0.02330097087378641), (18, 0.021359223300970873), (14, 0.021359223300970873), (19, 0.019417475728155338), (29, 0.019417475728155338), (23, 0.019417475728155338), (21, 0.017475728155339806), (24, 0.015533980582524271), (26, 0.013592233009708738), (28, 0.013592233009708738), (35, 0.011650485436893204), (30, 0.009708737864077669), (31, 0.009708737864077669), (27, 0.007766990291262136), (41, 0.005825242718446602), (33, 0.005825242718446602), (59, 0.00582524271

In [45]:
RDD_prob_distr_of_MCWs = prob_distr_of_most_common_words(RDD_word_counter, text_length)
print("PkMCW:\t", RDD_prob_distr_of_MCWs.take(30))

prob_the_most_common_word = prob_of_the_most_common_word(RDD_prob_distr_of_MCWs)
print("pMCW:\t", prob_the_most_common_word)

prob_the_most_common_word_x = prob_of_the_most_common_word_x(RDD_prob_distr_of_MCWs)
print("pMCWx:\t", prob_the_most_common_word_x)

prob_the = prob_of_The(RDD_prob_distr_of_MCWs)
print("pThe:\t", getValue(prob_the))

p_comma = prob_of_comma(sentences_data, text_length)
print("pComma:\t", p_comma)

PkMCW:	 [('the', 0.04494969293087678), ('i', 0.036717627074349925), ('and', 0.033320266562132494), ('to', 0.023650855873513656), ('of', 0.021298837057363126), ('a', 0.019730824513262774), ('my', 0.01620279628903698), ('that', 0.016072128577028617), ('in', 0.015941460865020254), ('was', 0.01528812230497844), ('said', 0.012152097216777734), ('as', 0.011106755520710831), ('he', 0.011106755520710831), ('at', 0.009930746112635568), ('but', 0.009669410688618842), ('you', 0.009277407552593753), ('for', 0.009146739840585392), ('me', 0.00862406899255194), ('it', 0.00862406899255194), ('had', 0.007840062720501764), ('with', 0.00718672416045995), ('not', 0.006533385600418136), ("o'conor", 0.006272050176401411), ('all', 0.006141382464393048), ('his', 0.005880047040376323), ('were', 0.00574937932836796), ('on', 0.005618711616359597), ('so', 0.004965373056317784), ('there', 0.004704037632301058), ('we', 0.004704037632301058)]
pMCW:	 ('i', 0.036717627074349925)
pMCWx:	 ('i', 0.036717627074349925)
pTh

In [47]:
MCW = prob_the_most_common_word[0]
dist_consec_MCW = distance_consec_appear(data, MCW)
print("adMCW:\t", sum(dist_consec_MCW)/len(dist_consec_MCW))
print("mdMCW:\t", min(dist_consec_MCW))
print("MdMCW:\t", max(dist_consec_MCW))

MCWx = prob_the_most_common_word_x[0]
dist_consec_MCWx = distance_consec_appear(data, MCWx)
print("adMCWx:\t", sum(dist_consec_MCWx)/len(dist_consec_MCWx))
print("mdMCWx:\t", min(dist_consec_MCWx))
print("MdMCWx:\t", max(dist_consec_MCWx))

dist_consec_the = distance_consec_appear(data, 'the')
print("adThe:\t", sum(dist_consec_the)/len(dist_consec_the))
print("mdThe:\t", min(dist_consec_the))
print("MdThe:\t", max(dist_consec_the))

dist_consec_comma = distance_consec_appear(sentences_data, ',')
print("adComma:", sum(dist_consec_comma)/len(dist_consec_comma))
print("mdComma:", min(dist_consec_comma))
print("MdComma:", max(dist_consec_comma))

adMCW:	 27.15357142857143
mdMCW:	 2
MdMCW:	 177
adMCWx:	 27.15357142857143
mdMCWx:	 2
MdMCWx:	 177
adThe:	 22.244897959183675
mdThe:	 2
MdThe:	 115
adComma: 14.713733075435202
mdComma: 1
MdComma: 92


In [47]:
metrics = generate_metrics("datasets/Anthony Trollope___The O'Conors of Castle Conor from Tales from all Countries.txt")
for key in metrics.keys():
    print(key, "-->", metrics[key], "\n")

Caricamento del file in memoria ... caricamento completato
Calcolo delle prime metriche, attendere ... calcolo completato
Caricamento del file in memoria ... caricamento completato
Calcolo di ulteriori metriche, attendere ... calcolo completato
vocabulary_size --> 1558 

text_length --> 7653 

V_T --> 0.20358029530902913 

entropy --> 8.605861060321123 

avg_sentence_len --> 14.862135922330097 

max_sentence_len --> 84 

min_sentence_len --> 1 

prob_distr_freq_sen --> [(7, 0.06407766990291262), (2, 0.05631067961165048), (9, 0.04660194174757282), (10, 0.04660194174757282), (17, 0.04271844660194175), (12, 0.040776699029126215), (8, 0.038834951456310676), (16, 0.038834951456310676), (1, 0.038834951456310676), (3, 0.038834951456310676), (15, 0.038834951456310676), (6, 0.036893203883495145), (11, 0.03495145631067961), (5, 0.03495145631067961), (13, 0.03300970873786408), (4, 0.031067961165048542), (20, 0.02912621359223301), (25, 0.02330097087378641), (22, 0.02330097087378641), (18, 0.021359

# Controllo salvataggio

In [8]:
from timeit import default_timer as timer

t = timer()
save_metrics("datasets/Anthony Trollope___The O'Conors of Castle Conor from Tales from all Countries.txt", "Anthony Trollope")
save_metrics("datasets/Anthony Trollope___George Walker At Suez.txt", "Anthony Trollope")
print("TIME: \n{} minuti".format(round((timer() - t)/60, 4)))

Caricamento del file in memoria ... caricamento completato
Calcolo delle prime metriche, attendere ... calcolo completato
Caricamento del file in memoria ... caricamento completato
Calcolo di ulteriori metriche, attendere ... calcolo completato
Salvataggio completato
Caricamento del file in memoria ... caricamento completato
Calcolo delle prime metriche, attendere ... calcolo completato
Caricamento del file in memoria ... caricamento completato
Calcolo di ulteriori metriche, attendere ... calcolo completato
Salvataggio completato
TIME: 
2.2314 minuti


In [32]:
a_metrics = author_metrics("Anthony Trollope")
for key in a_metrics.keys():
    print(key, "-->", a_metrics[key], "\n")

vocabulary_size --> (1586, 39.59797974644666) 

text_length --> (7877, 316.7838379715733) 

V_T --> (0.20140747884819438, 0.0030728265074600365) 

entropy --> (8.519433709418967, 0.1222267318058073) 

avg_sentence_len --> (18.620294480502064, 5.314838802515312) 

max_sentence_len --> (77, 9.899494936611665) 

min_sentence_len --> (1, 0.0) 

prob_distr_freq_sen --> [(8, (0.030467199485061415, 0.011833788324314906)), (16, (0.03599206136351445, 0.004020453725568525)), (24, (0.01881671404816821, 0.004642486188769695)), (32, (0.008847824920881832, 0.007020622496252209)), (48, (0.007876951134474066, 0.008393645372342592)), (56, (0.000970873786407767, 0.0013730228760903834)), (40, (0.0023520892560210263, 0.0005803107735962118)), (64, (0.0013812154696132596, 0.001953333649686595)), (9, (0.03849434103953227, 0.011465878879616658)), (17, (0.04207745534516977, 0.0009064985286895079)), (1, (0.023561122136995117, 0.021600456572747884)), (25, (0.018556562784959502, 0.006709606264651627)), (41, (0.00

## Bonus: confronto prestazioni tra funzioni di Spark e di libreria

In [48]:
from timeit import default_timer as timer

t = timer()
print("MSL:\t", RDD_sen_lengths.max())
print("ASL:\t", RDD_sen_lengths.mean())
print("mSL:\t", RDD_sen_lengths.min())
print("TIME: \n{}\n".format(round(timer() - t, 4)))

t = timer()
lengths = RDD_sen_lengths.collect()
print("MSL:\t", max(lengths))
print("ASL:\t", sum(lengths)/len(lengths))
print("mSL:\t", min(lengths))
print("TIME: \n{}\n".format(round(timer() - t, 4)))

MSL:	 84
ASL:	 14.862135922330097
mSL:	 1
TIME: 
14.5108

MSL:	 84
ASL:	 14.862135922330097
mSL:	 1
TIME: 
0.0296

